In [ ]:
# import necessary libraries
import pandas as pd
import os
import glob
import re

In [ ]:
# use glob to get all the csv file paths
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))
print(csv_files)

In [ ]:
survey_df = pd.read_csv('C:\\Users\\DaFon\\Desktop\\fulldata\\group-a\\survey-data-a.csv')
demo_df = pd.read_csv('C:\\Users\\DaFon\\Desktop\\fulldata\\group-a\\demographic-data-group-a.csv')

In [ ]:
survey_df.rename(columns = {'Please enter your prolific ID':'prolificId', 'How familiar are you with the driving law in Switzerland?':'familiarity', 'Is your 17-year-old relative from the USA allowed to drive a car in Switzerland?':'response', 'Why yes / why no? please elaborate on why you chose the above answer':'reason', 'How confident are you with your answer?':'confidence', 'How confident are you in your answer?':'confidence'}, inplace = True)
numberOfRowsSurveyDf = len(survey_df.index)

In [ ]:
# read all text file data that was tracked from the website
txt_files = glob.glob(os.path.join(path, "*.txt"))
columnNames = []
rows = []

for f in txt_files:
    file1 = open(f, 'r')
    lines = file1.readlines()
    
    counter = 0
    row = []
    for line in lines:
        if line.partition(":")[0] not in columnNames:
            columnNames.append(line.partition(":")[0])
        temp = line.partition(":")
        temp2 = temp[2:]
        temp3 = ''.join(temp2)
        temp4 = re.sub('\n', '', temp3)
        if not (counter == 1):
            temp4 = re.sub(' ', '', temp4)
        else:
            temp4 = temp4[1:]
        row.append(temp4)
        counter += 1
        
    rows.append(row)
    
rows.pop()
# rows is a list of lists, each list represents the data of one participant

# define column names that should be added for the data tracked from the website
columnNames.remove('ID')
columnNames.extend(['numberOfClicks','ClickedFeatured', 'timeSpentOnFeatured', 'clickedFirst', 'timeSpentOnFirst', 'clickedSecond', 'timeSpentOnSecond', 'clickedThird', 'timeSpentOnThird', 'clickedFourth', 'timeSpentOnFourth', 'clickedFifth', 'timeSpentOnFifth', 'clickedSixth', 'timeSpentOnSixth'])
print(columnNames)

In [ ]:
ids = [item[0] for item in rows]
searchTerms = [item[1] for item in rows]
totalSearchTimes = [item[2] for item in rows]
actions = [[item[3]] for item in rows]

In [ ]:
# find duplicate Focus values which can be deleted
toBeDeleted = []
for x in range(len(actions)):
    last = ''
    toBeDeletedList = []
    listOfActions = actions[x][0].split(',')
    for y in range(len(listOfActions)):
        if (listOfActions[y].split(':')[0] == 'Focus' and last == 'Focus'):
            toBeDeletedList.append(y-1)
        last = listOfActions[y].split(':')[0]
    toBeDeleted.append(toBeDeletedList)
            

# delete duplicate Focus values
actions2 = []
for x in range(len(actions)):
    listOfActions = actions[x][0].split(',')
    for y in range(len(listOfActions),-1,-1):
        if (y in toBeDeleted[x]):
            listOfActions.pop(y)
    actions2.append(listOfActions)
    

# delete Focus values, if there are no clicks on results
actions3 = []
for x in range(len(actions2)):
    listOfActions = actions2[x][0].split(',')
    onlyContainsFocus = True
    for y in range(len(listOfActions)-1,-1,-1):
        if (listOfActions[y].split(':')[0] != 'Focus'):
            onlyContainsFocus = False
            break
    if (onlyContainsFocus):
        actions3.append([''])
    else:
        actions3.append(actions2[x])

# now we have click and return pairs in actions3

# create clicked columns
numberOfClicks = []
clickedFeatured = []
clickedFirst = []
clickedSecond = []
clickedThird = []
clickedFourth = []
clickedFifth = []
clickedSixth = []
for x in range(len(actions3)):
    listOfActions = actions3[x]
    clicks = 0
    temp0 = False
    temp1 = False
    temp2 = False
    temp3 = False
    temp4 = False
    temp5 = False
    temp6 = False
    for y in range(len(listOfActions)):
        if (listOfActions[y].split(':')[0] == '0'):
            temp0 = True
            clicks += 1
        if (listOfActions[y].split(':')[0] == '1'):
            temp1 = True
            clicks += 1
        if (listOfActions[y].split(':')[0] == '2'):
            temp2 = True
            clicks += 1
        if (listOfActions[y].split(':')[0] == '3'):
            temp3 = True
            clicks += 1
        if (listOfActions[y].split(':')[0] == '4'):
            temp4 = True
            clicks += 1
        if (listOfActions[y].split(':')[0] == '5'):
            temp5 = True
            clicks += 1
        if (listOfActions[y].split(':')[0] == '6'):
            temp6 = True
            clicks += 1
    numberOfClicks.append(clicks)
    clickedFeatured.append(temp0)
    clickedFirst.append(temp1)
    clickedSecond.append(temp2)
    clickedThird.append(temp3)
    clickedFourth.append(temp4)
    clickedFifth.append(temp5)
    clickedSixth.append(temp6)
    
    
# create timeSpent columns
timeSpentOnFeatured = []
timeSpentOnFirst = []
timeSpentOnSecond = []
timeSpentOnThird = []
timeSpentOnFourth = []
timeSpentOnFifth = []
timeSpentOnSixth = []
for x in range(len(actions3)):
    listOfActions = actions3[x]
    timeSpent0 = 0.0
    timeSpent1 = 0.0
    timeSpent2 = 0.0
    timeSpent3 = 0.0
    timeSpent4 = 0.0
    timeSpent5 = 0.0
    timeSpent6 = 0.0
    for y in range(len(listOfActions)):
        if (listOfActions[y].split(':')[0] == '0' and y+1 < len(listOfActions)):
            timeSpent0 += float(listOfActions[y+1].split(':')[1]) - float(listOfActions[y].split(':')[1])
        if (listOfActions[y].split(':')[0] == '1' and y+1 < len(listOfActions)):
            timeSpent1 += float(listOfActions[y+1].split(':')[1]) - float(listOfActions[y].split(':')[1])
        if (listOfActions[y].split(':')[0] == '2' and y+1 < len(listOfActions)):
            timeSpent2 += float(listOfActions[y+1].split(':')[1]) - float(listOfActions[y].split(':')[1])
        if (listOfActions[y].split(':')[0] == '3' and y+1 < len(listOfActions)):
            timeSpent3 += float(listOfActions[y+1].split(':')[1]) - float(listOfActions[y].split(':')[1])
        if (listOfActions[y].split(':')[0] == '4' and y+1 < len(listOfActions)):
            timeSpent4 += float(listOfActions[y+1].split(':')[1]) - float(listOfActions[y].split(':')[1])
        if (listOfActions[y].split(':')[0] == '5' and y+1 < len(listOfActions)):
            timeSpent5 += float(listOfActions[y+1].split(':')[1]) - float(listOfActions[y].split(':')[1])
        if (listOfActions[y].split(':')[0] == '6' and y+1 < len(listOfActions)):
            timeSpent6 += float(listOfActions[y+1].split(':')[1]) - float(listOfActions[y].split(':')[1])
    timeSpentOnFeatured.append(timeSpent0)
    timeSpentOnFirst.append(timeSpent1)
    timeSpentOnSecond.append(timeSpent2)
    timeSpentOnThird.append(timeSpent3)
    timeSpentOnFourth.append(timeSpent4)
    timeSpentOnFifth.append(timeSpent5)
    timeSpentOnSixth.append(timeSpent6)

In [ ]:
behavior_data = [ids, searchTerms, totalSearchTimes, actions3, numberOfClicks, clickedFeatured, timeSpentOnFeatured, clickedFirst, timeSpentOnFirst, clickedSecond, timeSpentOnSecond, clickedThird, timeSpentOnThird, clickedFourth, timeSpentOnFourth, clickedFifth, timeSpentOnFifth, clickedSixth, timeSpentOnSixth]
 
# merge behavioral data from the website with data from the survey
for x in range(len(columnNames)):
    survey_df[columnNames[x]] = ""
    
for x in range(numberOfRowsSurveyDf):
    for y in range(len(behavior_data[0])):
        if (survey_df['prolificId'][x] == behavior_data[0][y]):
            for z in range(len(columnNames)):
                survey_df.at[x, columnNames[z]] = behavior_data[z+1][y]

output_df = survey_df

# merge behavioral data from the website and data from the survey with demographic data from prolific
columnNamesDemo = []

for col in demo_df.columns:
    if col not in output_df:
        columnNamesDemo.append(col)
        output_df[col] = ""

columnNamesDemo.remove('participant_id')
        
for x in range(numberOfRowsSurveyDf):
    for y in range(len(demo_df.index)):
        if (output_df['prolificId'][x] == demo_df['participant_id'][y]):
            for z in range(len(columnNamesDemo)):
                output_df.at[x, columnNamesDemo[z]] = demo_df[columnNamesDemo[z]][y]
                
survey_df.rename(columns = {'Unnamed: 6':'Problems'}, inplace = True)
display(output_df)
output_df.to_csv('output.csv', index = False, header=True)